In [1]:
import pandas as pd
import numpy as np
# import pyterrier_deepct
import pyterrier as pt
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

from xml.etree import ElementTree
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics2020.xml')
data = tree.getroot()
titles20 = []
titles21 = []
title_ids20 = []
title_ids21 = []
nums20 = []
nums21 = []
for topic in data:
    titles20.append(topic.find('title').text.lower()[:-1])
    num20 = topic.find('number').text
    nums20.append(num20)
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics.xml')
data = tree.getroot()    
for topic in data:
    titles21.append(topic.find('title').text.lower()[:-1])
    num21 = topic.find('number').text
    nums21.append(num21)
topics = pd.DataFrame()
topics['qid'] = nums20 + nums21
topics['query'] = titles20 + titles21

topics21 = pd.DataFrame()
topics21['qid'] = nums21
topics21['query'] = titles21

topics20 = pd.DataFrame()
topics20['qid'] = nums20
topics20['query'] = titles20

qrels_path=("/workspace/src/evaluate_targers/all_qrels.qrels")
qrels = pt.io.read_qrels(qrels_path)

qrels21_path=("/workspace/src/evaluate_targers/qrels2021.qrels")
qrels21 = pt.io.read_qrels(qrels21_path)

qrels20_path=("/workspace/src/evaluate_targers/qrels2020.qrels")
qrels20 = pt.io.read_qrels(qrels20_path)

docnos = qrels.docno
docnos21 = qrels21.docno
docnos20 = qrels20.docno

PyTerrier 0.8.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [2]:
import os
# rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/"
rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_original-100"

objective="ndcg_cut_5"
for i, (subdir, dirs, files) in enumerate(os.walk(rootdir)):
    for file in files:
        if(file == 'data.properties'):
            path = subdir + '/'+ file
            print('\n' + subdir)
            indexref = pt.IndexRef.of(path)
            index = pt.IndexFactory.of(indexref)
#             bm25_rm3 = (pt.BatchRetrieve(indexref, wmodel="BM25") >> 
#             pt.rewrite.RM3(indexref) >> 
#             pt.BatchRetrieve(indexref, wmodel="BM25"))
            bm25_for_qe = pt.BatchRetrieve(index, wmodel="BM25")
            rm3 = pt.rewrite.RM3(index, fb_terms=10, fb_docs=3)
            pipe_qe = bm25_for_qe >> rm3 >> bm25_for_qe
            
            param_map = {
                rm3 : {
                    "fb_terms" : list(range(4, 16, 2)),
                    "fb_docs" : list(range(4, 10, 2)),
                    "fb_lambda": [0.2,0.4,0.6,0.8,1.0]
                }
            }
            
            pipe_qe, parameters = pt.KFoldGridSearch(
                pipe_qe,
                param_map,
                [topics20, topics21],
                [qrels20, qrels21],
                objective
            )
            print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos)]], topics, qrels, 
                                names=['tuned_bm25_rm3_unjudged'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos20)]], topics20, qrels, 
                                names=['tuned_bm25_rm3_unjudged20'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos21)]], topics21, qrels, 
                                names=['tuned_bm25_rm3_unjudged21'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())            
            



/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_original-100
Fold 1
Best ndcg_cut_5 is 0.584666
Best setting is ['QueryExpansion(/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_original-100/data.properties,8,14,<org.terrier.querying.RM3 at 0x7f01ecf6de50 jclass=org/terrier/querying/RM3 jself=<LocalRef obj=0x558e0f9b51d8 at 0x7f01ed1053f0>>) fb_terms=12', 'QueryExpansion(/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_original-100/data.properties,8,14,<org.terrier.querying.RM3 at 0x7f01ecf6de50 jclass=org/terrier/querying/RM3 jself=<LocalRef obj=0x558e0f9b51d8 at 0x7f01ed1053f0>>) fb_docs=4', 'QueryExpansion(/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_original-100/data.properties,8,14,<org.terrier.querying.RM3 at 0x7f01ecf6de50 jclass=org/terrier/querying/RM3 jself=<LocalRe

/tmp/ipykernel_2391271/4205392178.py:35: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos)]], topics, qrels,
/tmp/ipykernel_2391271/4205392178.py:37: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos20)]], topics20, qrels,
/tmp/ipykernel_2391271/4205392178.py:39: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of 

In [3]:
import os
rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/"
folders = ['conclusions','pool_topics','topics','topics_conclusion','topics_conclusion_leakage','topics_leakage','pool_topics_conclusions','conclusions_leakage','pools_conclusions']
objective="ndcg_cut_5"
i = 0
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if(file == 'data.properties'):
            path = subdir + '/'+ file
            print('\n' + subdir)
            indexref = pt.IndexRef.of(path)
            index = pt.IndexFactory.of(indexref)
            bm25_for_qe = pt.BatchRetrieve(index, wmodel="BM25")
            rm3 = pt.rewrite.RM3(index, fb_terms=10, fb_docs=3)
            pipe_qe = bm25_for_qe >> rm3 >> bm25_for_qe
            
            param_map = {
                rm3 : {
                    "fb_terms" : list(range(4, 16, 2)),
                    "fb_docs" : list(range(4, 10, 2)),
                    "fb_lambda": [0.2,0.4,0.6,0.8,1.0]
                }
            }
            
            pipe_qe, parameters = pt.KFoldGridSearch(
                pipe_qe,
                param_map,
                [topics20, topics21],
                [qrels20, qrels21],
                objective
            )
            print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos)]], topics, qrels, names=[folders[i] + '_tuned_bm_rm3_unjudged'], 
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos20)]], topics20, qrels, names=[folders[i] + '_tuned_bm_rm3_unjudged20'], 
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos21)]], topics21, qrels, names=[folders[i] + '_tuned_bm_rm3_unjudged21'], 
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())            

            i+=1


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/conclusionspd_index
Fold 1
Best ndcg_cut_5 is 0.558365
Best setting is ['QueryExpansion(/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/conclusionspd_index/data.properties,8,14,<org.terrier.querying.RM3 at 0x7f019f2f29a0 jclass=org/terrier/querying/RM3 jself=<LocalRef obj=0x558e10679f30 at 0x7f019cdf02f0>>) fb_terms=6', 'QueryExpansion(/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/conclusionspd_index/data.properties,8,14,<org.terrier.querying.RM3 at 0x7f019f2f29a0 jclass=org/terrier/querying/RM3 jself=<LocalRef obj=0x558e10679f30 at 0x7f019cdf02f0>>) fb_docs=4', 'QueryExpansion(/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/conclusionspd_index/data.properties,8,14,<org.terrier.querying.RM3 at 0x7f019f2f29a0 jclass=org/terrier/querying/RM3 jself=<LocalRef obj=0x5

/tmp/ipykernel_2391271/4039785762.py:32: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos)]], topics, qrels, names=[folders[i] + '_tuned_bm_rm3_unjudged'],
/tmp/ipykernel_2391271/4039785762.py:34: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos20)]], topics20, qrels, names=[folders[i] + '_tuned_bm_rm3_unjudged20'],
/tmp/ipykernel_2391271/4039785762.py:36: FutureWar

\begin{tabular}{llrrr}
\toprule
{} &                               name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  conclusions\_tuned\_bm\_rm3\_unjudged &    0.720258 &      0.72585 &  0.757675 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                                 name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  conclusions\_tuned\_bm\_rm3\_unjudged20 &    0.802555 &     0.805013 &  0.770298 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                                 name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  conclusions\_tuned\_bm\_rm3\_unjudged21 &    0.673362 &     0.700574 &  0.745304 \\
\bottomrule
\end{tabular}


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/pool_topicspd_index
Fold 1
Best ndcg_cut_5 is 0.563813
Best setting is ['QueryExpansion(/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepC

/tmp/ipykernel_2391271/4039785762.py:32: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos)]], topics, qrels, names=[folders[i] + '_tuned_bm_rm3_unjudged'],
/tmp/ipykernel_2391271/4039785762.py:34: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos20)]], topics20, qrels, names=[folders[i] + '_tuned_bm_rm3_unjudged20'],
/tmp/ipykernel_2391271/4039785762.py:36: FutureWar

\begin{tabular}{llrrr}
\toprule
{} &                               name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  pool\_topics\_tuned\_bm\_rm3\_unjudged &    0.716481 &     0.729354 &  0.756732 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                                 name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  pool\_topics\_tuned\_bm\_rm3\_unjudged20 &    0.785007 &     0.805431 &  0.763596 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                                 name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  pool\_topics\_tuned\_bm\_rm3\_unjudged21 &    0.699341 &     0.719756 &  0.750005 \\
\bottomrule
\end{tabular}


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/topicspd_index
Fold 1
Best ndcg_cut_5 is 0.532393
Best setting is ['QueryExpansion(/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_

...

/tmp/ipykernel_2391271/4039785762.py:32: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos)]], topics, qrels, names=[folders[i] + '_tuned_bm_rm3_unjudged'],
/tmp/ipykernel_2391271/4039785762.py:34: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos20)]], topics20, qrels, names=[folders[i] + '_tuned_bm_rm3_unjudged20'],
/tmp/ipykernel_2391271/4039785762.py:36: FutureWar

\begin{tabular}{llrrr}
\toprule
{} &                                           name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  pool\_topics\_conclusions\_tuned\_bm\_rm3\_unjudged &    0.713689 &     0.726816 &  0.758526 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                                             name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  pool\_topics\_conclusions\_tuned\_bm\_rm3\_unjudged20 &    0.797176 &     0.807338 &  0.768932 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                                             name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  pool\_topics\_conclusions\_tuned\_bm\_rm3\_unjudged21 &    0.673796 &     0.703676 &  0.748328 \\
\bottomrule
\end{tabular}


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/conclusions_leakage/pd_index
Fold 1
Best ndcg_cut_5 is 0.550328
Best setting is ['QueryExpan

/tmp/ipykernel_2391271/4039785762.py:32: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos)]], topics, qrels, names=[folders[i] + '_tuned_bm_rm3_unjudged'],
/tmp/ipykernel_2391271/4039785762.py:34: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos20)]], topics20, qrels, names=[folders[i] + '_tuned_bm_rm3_unjudged20'],
/tmp/ipykernel_2391271/4039785762.py:36: FutureWar

\begin{tabular}{llrrr}
\toprule
{} &                                       name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &    bpref \\
\midrule
0 &  conclusions\_leakage\_tuned\_bm\_rm3\_unjudged &    0.728712 &     0.730277 &  0.75198 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                                         name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  conclusions\_leakage\_tuned\_bm\_rm3\_unjudged20 &    0.808583 &     0.807233 &  0.764244 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                                         name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &    bpref \\
\midrule
0 &  conclusions\_leakage\_tuned\_bm\_rm3\_unjudged21 &     0.68259 &     0.705607 &  0.73996 \\
\bottomrule
\end{tabular}


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/pools_conclusionspd_index
Fold 1
Best ndcg_cut_5 is 0.539135
Best setting is ['QueryExpansion(/mnt/ceph/storage/data-in-pro

/tmp/ipykernel_2391271/4039785762.py:32: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos)]], topics, qrels, names=[folders[i] + '_tuned_bm_rm3_unjudged'],
/tmp/ipykernel_2391271/4039785762.py:34: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos20)]], topics20, qrels, names=[folders[i] + '_tuned_bm_rm3_unjudged20'],
/tmp/ipykernel_2391271/4039785762.py:36: FutureWar

In [5]:
import os
rootdir = "/workspace/src/grid_search_right_qrels/super_folder/"
folders = ['marco-pretrained']
objective="ndcg_cut_5"
i = 0
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if(file == 'data.properties'):
            path = subdir + '/'+ file
            print('\n' + subdir)
            indexref = pt.IndexRef.of(path)
            index = pt.IndexFactory.of(indexref)
            bm25_for_qe = pt.BatchRetrieve(index, wmodel="BM25")
            rm3 = pt.rewrite.RM3(index, fb_terms=10, fb_docs=3)
            pipe_qe = bm25_for_qe >> rm3 >> bm25_for_qe
            
            param_map = {
                rm3 : {
                    "fb_terms" : list(range(4, 16, 2)),
                    "fb_docs" : list(range(4, 10, 2)),
                    "fb_lambda": [0.2,0.4,0.6,0.8,1.0]
                }
            }
            
            pipe_qe, parameters = pt.KFoldGridSearch(
                pipe_qe,
                param_map,
                [topics20, topics21],
                [qrels20, qrels21],
                objective
            )
            print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos)]], topics, qrels, names=[folders[i] + '_tuned_bm_rm3_unjudged'], 
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos20)]], topics20, qrels, names=[folders[i] + '_tuned_bm_rm3_unjudged20'], 
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos21)]], topics21, qrels, names=[folders[i] + '_tuned_bm_rm3_unjudged21'], 
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())            

            i+=1


/workspace/src/grid_search_right_qrels/super_folder/
Fold 1
Best ndcg_cut_5 is 0.452738
Best setting is ['QueryExpansion(/workspace/src/grid_search_right_qrels/super_folder/data.properties,8,14,<org.terrier.querying.RM3 at 0x7f0277ecd450 jclass=org/terrier/querying/RM3 jself=<LocalRef obj=0x558e10679ee0 at 0x7f019cc911b0>>) fb_terms=10', 'QueryExpansion(/workspace/src/grid_search_right_qrels/super_folder/data.properties,8,14,<org.terrier.querying.RM3 at 0x7f0277ecd450 jclass=org/terrier/querying/RM3 jself=<LocalRef obj=0x558e10679ee0 at 0x7f019cc911b0>>) fb_docs=8', 'QueryExpansion(/workspace/src/grid_search_right_qrels/super_folder/data.properties,8,14,<org.terrier.querying.RM3 at 0x7f0277ecd450 jclass=org/terrier/querying/RM3 jself=<LocalRef obj=0x558e10679ee0 at 0x7f019cc911b0>>) fb_lambda=0.4']
Fold 2
Best ndcg_cut_5 is 0.285510
Best setting is ['QueryExpansion(/workspace/src/grid_search_right_qrels/super_folder/data.properties,8,14,<org.terrier.querying.RM3 at 0x7f0277ecd450 jcla

/tmp/ipykernel_2391271/2189430013.py:32: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos)]], topics, qrels, names=[folders[i] + '_tuned_bm_rm3_unjudged'],
/tmp/ipykernel_2391271/2189430013.py:34: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos20)]], topics20, qrels, names=[folders[i] + '_tuned_bm_rm3_unjudged20'],
/tmp/ipykernel_2391271/2189430013.py:36: FutureWar